# P-Median
<img src='../imgs/p_median.bmp' height=300></img>

where:  
**m** is the number of customers,  
**n** is the number of potential facility locations,  
**p** is the number of facilities to be opened,  
**cᵢⱼ** is the cost of serving customer *i* from facility *j*,  
**xᵢⱼ** is a binary decision variable indicating whether customer *i* is served by facility *j*, and  
**yⱼ** is a binary decision variable indicating whether facility *j* is opened.

In [1]:
# import matrix
import pandas as pd
from pulp import *
import numpy as np

df = pd.read_csv('../data/distance.csv', delimiter=';', index_col=0)
df


,Kurudampalayam,Ashokapuram,Peelamedu,Vellalore,KovaiPudur,Vadavalli
Kurudampalayam,0.0,5.6,17.4,21.8,19.4,11.4
Ashokapuram,5.6,0.0,11.6,19.6,21.7,13.0
Peelamedu,17.4,11.6,0.0,8.0,18.3,16.8
Vellalore,21.8,19.6,8.0,0.0,16.9,20.1
KovaiPudur,19.4,21.7,18.3,16.9,0.0,13.6
Vadavalli,11.4,13.0,16.8,20.1,13.6,0.0


**Definindo variaveis**

In [2]:
location = df.columns.values
clients = df.index.values 
w = [1,1,1,1,1,1]
m = df.shape[0] # Nº de clientes
n = df.shape[1] # Nº de potenciais facilidades
p= 3 # Nº de facilidades   
c = df.values


In [3]:
# Define the problem
prob = LpProblem("p-median", LpMinimize)

In [4]:
# Define decision variables
x = LpVariable.dicts("x", [(i,j) for i in range(m) for j in range(n)], 0, 1, LpBinary)
y = LpVariable.dicts("y", [j for j in range(n)], 0, 1, LpBinary)

In [5]:
# Define objective function
prob += lpSum([c[i][j] * x[(i,j)] * w[i] for i in range(m) for j in range(n)])

In [6]:
# Define constraints
for i in range(m):
    prob += lpSum([x[(i,j)] for j in range(n)]) == 1

for i in range(m):
    for j in range(n):
        prob += x[(i,j)] <= y[j]
                 
prob += lpSum([y[j] for j in range(n)]) == p

The first constraint, <code> for i in range(m): prob += lpSum([x[(i,j)] for j in range(n)]) == 1,</code> ensures that each customer is assigned to exactly one facility. It is expressed by summing over all facilities j and requiring that each customer i is served by exactly one of them.  

The second constraint, `for i in range(m): for j in range(n): prob += x[(i,j)] <= y[j]`, ensures that if a facility is open, then the customers served by that facility should be assigned to it. This is expressed by saying that if x[i,j] = 1 (facility j serves customer i), then y[j] must also be 1 (facility j is open).  

The third constraint, `prob += lpSum([y[j] for j in range(n)]) == p`, limits the number of facilities that can be open to p. It is expressed by summing over all facilities j and requiring that the total number of open facilities is exactly equal to p.  

Together, these constraints ensure that the objective function is minimized subject to the conditions that each customer is served by exactly one facility, each facility can serve a limited number of customers, and the total number of open facilities is limited top.



In [7]:
# Solve the problem using branch-and-bound algorithm
prob.solve(PULP_CBC_CMD(gapRel=0.0, threads=1, timeLimit=600))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/fabio/PycharmProjects/P-Median/env/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/cc481defb04a41c48ff425696b3c9ae7-pulp.mps -sec 600 -ratio 0.0 -threads 1 -timeMode elapsed -branch -printingOptions all -solution /tmp/cc481defb04a41c48ff425696b3c9ae7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 277 RHS
At line 321 BOUNDS
At line 364 ENDATA
Problem MODEL has 43 rows, 42 columns and 114 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 600
ratioGap was changed from 0 to 0
threads was changed from 0 to 1
Option for timeMode changed from cpu to elapsed
Continuous objective value is 25 - 0.00 seconds
Cgl0004I processed model has 43 rows, 42 columns (42 integer (42 of which binary)) and 114 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 0 integers unsatisfied s

1

In [8]:
# Print results
facilities = []
X = np.zeros([6,6])
times = []
print("Optimal objective value:", value(prob.objective))
for j in range(n):
    if y[j].value() > 0.5:
        facilities.append(location[j])
        print("Facility", j, "is located.", location[j])
        for i in range(m):
            if x[(i,j)].value() > 0.5:
                X[i,j]=1
                times.append(df.iloc[i,j])
                print("- Customer", i, "is served.", clients[i])

Optimal objective value: 25.0
Facility 0 is located. Kurudampalayam 
- Customer 0 is served. Kurudampalayam
- Customer 1 is served. Ashokapuram
- Customer 5 is served. Vadavalli
Facility 3 is located. Vellalore
- Customer 2 is served. Peelamedu
- Customer 3 is served. Vellalore
Facility 4 is located. KovaiPudur
- Customer 4 is served. KovaiPudur


In [9]:
print('Facilidades abertas:\n', facilities)
print('matriz de variaveis de decisão:\n', X)

Facilidades abertas:
 ['Kurudampalayam ', 'Vellalore', 'KovaiPudur']
matriz de variaveis de decisão:
 [[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [266]:
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        if X[i,j] == 0:
            df.iloc[i,j] = X[i,j]

In [267]:
df

,Kurudampalayam,Ashokapuram,Peelamedu,Vellalore,KovaiPudur,Vadavalli
Kurudampalayam,0.0,0.0,0.0,0.0,0.0,0.0
Ashokapuram,5.6,0.0,0.0,0.0,0.0,0.0
Peelamedu,0.0,0.0,0.0,8.0,0.0,0.0
Vellalore,0.0,0.0,0.0,0.0,0.0,0.0
KovaiPudur,0.0,0.0,0.0,0.0,0.0,0.0
Vadavalli,11.4,0.0,0.0,0.0,0.0,0.0


In [268]:
times

[0.0, 5.6, 11.4, 8.0, 0.0, 0.0]